Extract Data from jobvision..

In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta
from urllib.parse import urlparse, parse_qs

functions

In [13]:
def get_info_from_site(pages, base_url, driver, start_from=1):
    jobs_list = []
    now = datetime.now()
    empty_value_count = 0
    for page in range(start_from, pages + 1):
        try:
            url = base_url + str(page)
            driver.get(url)

            jobs = WebDriverWait(driver, 15).until(
                EC.visibility_of_element_located((By.TAG_NAME, "job-card-list"))
            )

            jobs_element_list = jobs.find_elements(By.TAG_NAME, "job-card")

            try:
                for job in jobs_element_list:

                    # Get information about job postings from post

                    title = job.find_element(By.CLASS_NAME, "job-card-title")

                    parent = title.find_element(By.XPATH, "..")
                    childs_list = parent.find_elements(By.XPATH, "./*")

                    company = childs_list[1]

                    job_info_element = childs_list[2]

                    if len(childs_list) == 5:
                        facility = childs_list[3].text
                    else:
                        facility = ""
                    shipping_time = childs_list[-1]

                    job_info_list = job_info_element.find_elements(By.TAG_NAME, "span")

                    if len(job_info_list) == 3:
                        region, _, salary = job_info_list
                        salary = salary.text
                    else:
                        region = job_info_list[0]
                        salary = ""

                    extractec_num = "".join(
                        filter(lambda x: x.isnumeric(), shipping_time.text)
                    )
                    if extractec_num != "":
                        release_time = now - timedelta(days=int(extractec_num))
                    else:
                        release_time = now - timedelta(days=0)

                    # Go to any of the job position

                    title.click()

                    get_job_section = WebDriverWait(driver, 10).until(
                        EC.visibility_of_element_located(
                            (By.TAG_NAME, "app-job-specification")
                        )
                    )

                    job_info_list = get_job_section.find_elements(
                        By.CLASS_NAME, "text-muted"
                    )
                    working_hours, type_of_cooperation = job_info_list[:2]
                    try:
                        requirement_list = get_job_section.find_elements(
                            By.TAG_NAME, "app-tag"
                        )
                        requirement_software_list = [i.text for i in requirement_list]
                    except:
                        requirement_software_list = []

                    jobs_list.append(
                        {
                            "shippingDate": release_time.date(),
                            "title": title.text,
                            "company": company.text,
                            "salary": salary,
                            "province": region.text,
                            "workingHours": working_hours.text,
                            "typeOfCooperation": type_of_cooperation.text,
                            "facility": facility,
                            "tagList": requirement_software_list,
                        }
                    )
            except ValueError as e:
                jobs_list.append(
                    {
                        "shippingDate": "",
                        "title": "",
                        "company": "",
                        "salary": "",
                        "province": "",
                        "workingHours": "",
                        "typeOfCooperation": "",
                        "facility": "",
                        "tagList": "",
                    }
                )
                empty_value = empty_value + 1
                if empty_value_count == 8000:
                    print(e)

        except:
            current_url = driver.current_url
            parsed_url = urlparse(current_url)
            query_params = parse_qs(parsed_url.query)
            current_page = int(query_params.get("page")[0])
            get_info_from_site(current_url, base_url, driver, current_page)

    return jobs_list


def find_last_page(base_url, driver):
    driver.get(base_url + "1")

    last_page = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "pagination-last"))
    )

    last_page.click()

    current_url = driver.current_url
    parsed_url = urlparse(current_url)
    query_params = parse_qs(parsed_url.query)
    pages = int(query_params.get("page")[0])
    return pages

In [ ]:
options = Options()
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)
base_url = "https://jobvision.ir/jobs?page="

pages = find_last_page(base_url, driver)

jobs_list = get_info_from_site(pages, base_url, driver)

driver.quit()

df = pd.DataFrame(jobs_list)

df.to_csv("job_data_optimized.csv", index=False)

آخرین


TimeoutException: Message: 
